In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import time
import matplotlib as plt
import dateutil.parser
import random

In [2]:
#URLs of the 5 pages of top 1000 domestic gross movies of all time
top0_200= ['https://www.boxofficemojo.com/chart/top_lifetime_gross/']
top201_400 = ['https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=200']
top401_600 = ['https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=400']
top601_800 = ['https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=600']
top801_1000 = ['https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=800']

In [3]:
#have to do one page at a time because it takes 10 mins for each page to fetch all features
def getmovielinks(url_list):
    movie_links=[]
    for url in url_list:
        response=requests.get(url)
        soup_movie=bs(response.text, 'lxml')
        #grab movie links
        for x in soup_movie.find_all('a', class_='a-link-normal'):
            movie_links.append(x.get('href'))
    return movie_links

In [4]:
#saving partial links into a variable movielinks
movielinks1=getmovielinks(top0_200)
movielinks2=getmovielinks(top201_400)
movielinks3=getmovielinks(top401_600)
movielinks4=getmovielinks(top601_800)
movielinks5=getmovielinks(top801_1000)

In [5]:
#take movielinks and grab only movie URLs from movielinks
#and append to a list of final movie_urls
def mojomovies(movielinks):
    movie_urls=[]
    for link in movielinks:
        if "title" in link:
            final='https://www.boxofficemojo.com'+link
            movie_urls.append(final)
            
    return movie_urls

In [6]:
#store list of urls in movie_num variable
movie_num1=mojomovies(movielinks1)
movie_num2=mojomovies(movielinks2)
movie_num3=mojomovies(movielinks3)
movie_num4=mojomovies(movielinks4)
movie_num5=mojomovies(movielinks5)

In [ ]:
#testing to see what it looks like
print(movie_num1)

In [7]:
#checking to see how many URL links gathered
#we sould have 200 URLs in each! 
print(len(movie_num1), len(movie_num2), len(movie_num3), len(movie_num4), len(movie_num5))

200 200 200 200 200


In [ ]:
#getting data from pages and saving into dataframe 
def movie_info(movie_num): 
    #lists to hold values from webpages
    Titles=[]
    Domestic_Gr=[]
    Dom_Open=[]
    Budget=[]
    Rel_Date=[]
    Runtime=[]
    Genre=[]
    Rating=[]
    Distributor=[]
    
#loop through each page  
    for i in movie_num:       
        response = requests.get(i)
        
        ##DO THIS: ADD PAUSES HERE TO MIMIC HUMAN USER
        #import time 
        #import fakeuser 

        #saving imported text as variable
        page = response.text

        #parser
        soup = bs(page, "lxml")
    
    
        #GET TITLE FROM EACH PAGE
        try:  
            title=soup.find('h1', class_='a-size-extra-large').text
            Titles.append(title)
        except Exception as error:
            Titles.append(np.nan) 

        #GET DOMESTIC GROSS              
        try:
            domestic_gross=soup.find('span', class_='a-size-medium a-text-bold').text.split()[0]
            Domestic_Gr.append(domestic_gross)
        except Exception as error:
            Domestic_Gr.append(np.nan)
            
        #DOMESTIC OPENING
        try:
            domestic_open = soup.find('span', text = 'Domestic Opening').find_next('span', class_='money').text
            Dom_Open.append(domestic_open)
        except Exception as error:
            Dom_Open.append(np.nan)

        #BUDGET
        try:
            budget = soup.find('span', text = 'Budget').find_next('span', class_='money').text
            Budget.append(budget)
        except Exception as error:
            Budget.append(np.nan)
            
        #RELEASE DATE
        try:
            release_date=soup.find('span', text = 'Earliest Release Date').find_next('span').text
            Rel_Date.append(release_date)
        except Exception as error:
            Rel_Date.append(np.nan)
    
        #RUNTIME
        try:
            runtime=soup.find('span', text = 'Running Time').find_next('span').text
            Runtime.append(runtime)
        except Exception as error:
            Runtime.append(np.nan)
                  
        #GENRE
        try:
            genre=soup.find('span', text = 'Genres').find_next('span').text
            Genre.append(genre)
        except Exception as error:
            Genre.append(np.nan)
              
        #RATING
        try: 
            rating=soup.find('span', text = 'MPAA').find_next('span').text
            Rating.append(rating)
        except Exception as error:
            Rating.append(np.nan)
            
         #DISTRIBUTOR
        try:
            distributor=soup.find('span', text='Domestic Distributor').find_next('span').text
            Distributor.append(distributor)
        except Exception as error:
            Distributor.append(np.nan)

    movies_dict={'Title':Titles,'Domestic_Gr':Domestic_Gr,'Dom_Open':Dom_Open, 
                 'Budget':Budget,'Rel_Date':Rel_Date, 'Runtime':Runtime, 
                 'Genre':Genre,'Rating':Rating, 'Distributor': Distributor}
        
    movies_data=pd.DataFrame(movies_dict)
    
    return movies_data
    

In [ ]:
df = movie_info(movie_num1)
df.to_pickle("df.pkl")

In [ ]:
df2 = movie_info(movie_num2)
df2.to_pickle("df2.pkl")

In [ ]:
df3 = movie_info(movie_num3)
df3.to_pickle("df3.pkl")

In [ ]:
df4 = movie_info(movie_num4)
df4.to_pickle("df4.pkl")

In [ ]:
df5 = movie_info(movie_num5)
df5.to_pickle("df5.pkl")

In [8]:
df = pd.read_pickle("df.pkl")
df2 = pd.read_pickle("df2.pkl")
df3 = pd.read_pickle("df3.pkl")
df4 = pd.read_pickle("df4.pkl")
df5 = pd.read_pickle("df5.pkl")  

In [ ]:
top1000_mojo = pd.concat([df, df2, df3, df4, df5], axis=0).reset_index()

In [ ]:
top1000_mojo.head()

In [ ]:
top1000_mojo.to_pickle("top1000_mojo.pkl")

In [9]:
top1000_mojo = pd.read_pickle("top1000_mojo.pkl")  

In [10]:
top1000_mojo.info()
#1000 movies, wahoo!!!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        1000 non-null   int64 
 1   Title        1000 non-null   object
 2   Domestic_Gr  1000 non-null   object
 3   Dom_Open     969 non-null    object
 4   Budget       807 non-null    object
 5   Rel_Date     1000 non-null   object
 6   Runtime      1000 non-null   object
 7   Genre        1000 non-null   object
 8   Rating       876 non-null    object
 9   Distributor  999 non-null    object
dtypes: int64(1), object(9)
memory usage: 78.2+ KB


In [ ]:
###TESTING OUT FUNCTION BELOW TO MAKE SURE IT WORKS###### MAIN CHARACTER
#url= 'https://www.boxofficemojo.com/title/tt0281373/credits/?ref_=bo_tt_tab#tabs'
#Landingpage = 'https://www.boxofficemojo.com/title/tt0281373/?ref_=bo_tt_tab#tabs'
#response=requests.get(url)
#soup=bs(response.text,'html5lib')
#Cast_sex = []
#table = soup.find(id='principalCast').next.next.next.next.next.next.next.text
#table

In [ ]:
###TESTING OUT FUNCTION BELOW TO MAKE SURE IT WORKS###### DIRECTOR
#url= 'https://www.boxofficemojo.com/title/tt2488496/credits/?ref_=bo_tt_tab#tabs'
#response=requests.get(url)
#soup=bs(response.text,'html5lib')
#Director = []
#table = soup.find(text='Director').parent.parent.text
#table

In [ ]:
print(movielinks1, movielinks2, movielinks3, movielinks4, movielinks5)

In [12]:
print(movie_num1, movie_num2, movie_num3, movie_num4, movie_num5)

In [13]:
#Function to import main character of each top 1000 movies, HOWEVER,
#First have to change the URL in a function/loop for each of the movies' links
def mojo_cast(movielinks):
    movie_cast_urls=[]
    #For URL https://www.boxofficemojo.com/title/tt2488496/?ref_=bo_tt_tab#tabs
    #We want https://www.boxofficemojo.com/title/tt2488496/credits/?ref_=bo_tt_tab#tabs
    insert = "/credits"
    index = 45
    for link in movielinks:
        final = link[:index] + insert + link[index:]
        movie_cast_urls.append(final)   
    return movie_cast_urls

In [15]:
credits1 = mojo_cast(movie_num1)
credits2 = mojo_cast(movie_num2)
credits3 = mojo_cast(movie_num3)
credits4 = mojo_cast(movie_num4)
credits5 = mojo_cast(movie_num5)

In [16]:
print(len(credits1), len(credits2), len(credits3), len(credits4), len(credits5))
#should be 200 in each

200 200 200 200 200


In [ ]:
#checking to see if all new links came out right
print(credits1, credits2, credits3, credits4, credits5)

In [17]:
#Function to fetch the first cast member's name on each of the movies' pages
#To also fetch the first director listed on each of the movies' pages
def cast_name(credits):
    Cast_sex = []
    Titles = []
    Director = []
    for i in credits:
        response = requests.get(i)
        soup=bs(response.text,'html5lib')
        #GET TITLE FROM EACH PAGE
        try:  
            title=soup.find('h1', class_='a-size-extra-large').text
            Titles.append(title)
        except Exception as error:
            Titles.append(np.nan) 
            
        #GET FIRST CAST MEMBER LISTED FROM EACH PAGE
        try:
            table = soup.find(id='principalCast').next.next.next.next.next.next.next.text
            Cast_sex.append(table)
        except Exception as error:
            Cast_sex.append(np.nan)
            
        #GET FIRST DIRECTOR LISTED FROM EACH PAGE
        try:
            table = soup.find(text="Director").parent.parent.text
            Director.append(table)
        except Exception as error:
            Director.append(np.nan)

    name_dict = {"Title": Titles, "Cast": Cast_sex, "Director": Director}
    
    cast_data=pd.DataFrame(name_dict)
    
    return cast_data

In [ ]:
#have to apply this function one by one due to processing
df_cast_1 = cast_name(credits1)

In [ ]:
df_cast_2 = cast_name(credits2)

In [ ]:
df_cast_3 = cast_name(credits3)

In [ ]:
df_cast_4 = cast_name(credits4)

In [ ]:
df_cast_5 = cast_name(credits5)

In [ ]:
print(df_cast_1, df_cast_2, df_cast_3, df_cast_4, df_cast_5)

In [20]:
print(len(df_cast_1), len(df_cast_2), len(df_cast_3), len(df_cast_4), len(df_cast_5))
#200 in each, great!

200 200 200 200 200


In [ ]:
df_cast_1.to_pickle("df_cast_1.pkl")
df_cast_2.to_pickle("df_cast_2.pkl")
df_cast_3.to_pickle("df_cast_3.pkl")
df_cast_4.to_pickle("df_cast_4.pkl")
df_cast_5.to_pickle("df_cast_5.pkl")

In [156]:
df_cast_1 = pd.read_pickle("df_cast_1.pkl")
df_cast_2 = pd.read_pickle("df_cast_2.pkl")
df_cast_3 = pd.read_pickle("df_cast_3.pkl")
df_cast_4 = pd.read_pickle("df_cast_4.pkl")
df_cast_5 = pd.read_pickle("df_cast_5.pkl")  

In [157]:
top1000_cast_direct = pd.concat([df_cast_1, df_cast_2, df_cast_3, df_cast_4, df_cast_5], axis=0).reset_index()
top1000_cast_direct
#there should be 1000 rows, great!
#there are missing values, lets look a little deeper

,index,Title,Cast,Director
0,0,Star Wars: Episode VII - The Force Awakens (2015),Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,1,Avengers: Endgame (2019),Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,2,NaN,NaN,NaN
3,3,Avatar (2009),Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,4,Black Panther (2018),Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director
...,...,...,...,...
995,195,The Notebook (2004),Gena Rowlands\n \n Allie CalhounSee ...,Nick Cassavetes\n \n Director
996,196,The Talented Mr. Ripley (1999),Matt Damon\n \n Tom RipleySee more,Anthony Minghella\n \n Director
997,197,School of Rock (2003),Jack Black\n \n Dewey FinnSee more,Richard Linklater\n \n Director
998,198,Police Academy (1984),Steve Guttenberg\n \n Carey MahoneyS...,Hugh Wilson\n \n Director


In [158]:
#diving deeper into missing values
top1000_cast_direct[top1000_cast_direct.isna().any(axis=1)]
#okay we have 5 missing values. Based on index, we can add +1 to index to see which ranking they
#are based on https://www.boxofficemojo.com/chart/top_lifetime_gross/?ref_=bo_cso_ac top 1000 domestric rankings

,index,Title,Cast,Director
2,2,NaN,NaN,NaN
241,41,NaN,NaN,NaN
705,105,NaN,NaN,NaN
848,48,NaN,NaN,NaN
988,188,NaN,NaN,NaN


In [159]:
#missing movies that I need to manually add are...
#ranking 3, 242, 706, 849, and 989 on list.
top1000_cast_direct.loc[2, ["Title"]] = "Spider-Man: No Way Home (2021)"
top1000_cast_direct.loc[2, ["Cast"]] = "Tom Holland"
top1000_cast_direct.loc[2, ["Director"]] = "Jon Watts"

top1000_cast_direct.loc[241, ["Title"]] = "Sonic the Hedgehog 2 (2022)"
top1000_cast_direct.loc[241, ["Cast"]] = "James Marsden"
top1000_cast_direct.loc[241, ["Director"]] = "Jeff Fowler"

top1000_cast_direct.loc[848, ["Title"]] = "Halloween Kills (2021)"
top1000_cast_direct.loc[848, ["Cast"]] = "Jamie Lee Curtis"
top1000_cast_direct.loc[848, ["Director"]] = "David Gordon Green"

top1000_cast_direct.loc[988, ["Title"]] = "Scream (2022)"
top1000_cast_direct.loc[988, ["Cast"]] = "Neve Campbell"
top1000_cast_direct.loc[988, ["Director"]] = "Matt Bettinelli-Olpin"

top1000_cast_direct.loc[705, ["Title"]] = "The Lost City (2022)"
top1000_cast_direct.loc[705, ["Cast"]] = "Sandra Bullock"
top1000_cast_direct.loc[705, ["Director"]] = "Aaron Nee"

In [160]:
#sanity check to make sure all are entered right
top1000_cast_direct.loc[[2, 241, 705, 848, 988]]

,index,Title,Cast,Director
2,2,Spider-Man: No Way Home (2021),Tom Holland,Jon Watts
241,41,Sonic the Hedgehog 2 (2022),James Marsden,Jeff Fowler
705,105,The Lost City (2022),Sandra Bullock,Aaron Nee
848,48,Halloween Kills (2021),Jamie Lee Curtis,David Gordon Green
988,188,Scream (2022),Neve Campbell,Matt Bettinelli-Olpin


In [161]:
#sanity check again
top1000_cast_direct[top1000_cast_direct.isna().any(axis=1)]
#no missing values...HOORAY!

,index,Title,Cast,Director


In [162]:
top1000_cast_direct['Title'].value_counts()

Star Wars: Episode VII - The Force Awakens (2015)    1
Superman II (1980)                                   1
Star Trek IV: The Voyage Home (1986)                 1
The 40-Year-Old Virgin (2005)                        1
Crocodile Dundee II (1988)                           1
                                                    ..
Jason Bourne (2016)                                  1
Grown Ups (2010)                                     1
The Waterboy (1998)                                  1
Ice Age 4: Continental Drift (2012)                  1
Snow Dogs (2002)                                     1
Name: Title, Length: 1000, dtype: int64

In [163]:
top1000_cast_direct.to_pickle("top1000_cast_direct.pkl")

In [164]:
# Merging the two complete dataframes based on TITLE of movie
top1000 = (pd.merge(top1000_mojo, top1000_cast_direct, on="Title"))

In [166]:
top1000
#should be 1000 rows

,index_x,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,index_y,Cast,Director
0,0,Star Wars: Episode VII - The Force Awakens (2015),"$936,662,225","$247,966,675","$245,000,000","December 16, 2015\n (EMEA, APAC)",2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,0,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,1,Avengers: Endgame (2019),"$858,373,000","$357,115,007","$356,000,000","April 24, 2019\n (21 markets)",3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,1,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,2,Spider-Man: No Way Home (2021),"$804,761,122","$260,138,569",NaN,"December 15, 2021\n (EMEA, APAC)",2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,2,Tom Holland,Jon Watts
3,3,Avatar (2009),"$760,507,625","$77,025,481","$237,000,000","December 16, 2009\n (LATAM, EMEA)",2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,3,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,4,Black Panther (2018),"$700,426,566","$202,003,951",NaN,"February 13, 2018\n (APAC, EMEA)",2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,4,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,195,The Notebook (2004),"$81,417,274","$13,464,745","$29,000,000","June 25, 2004\n (EMEA, Domestic)",2 hr 3 min,Drama\n \n Romance,PG-13,New Line CinemaSee full company information\n\n,195,Gena Rowlands\n \n Allie CalhounSee ...,Nick Cassavetes\n \n Director
996,196,The Talented Mr. Ripley (1999),"$81,298,265","$12,738,237","$40,000,000","December 25, 1999\n (Domestic)",2 hr 19 min,Crime\n \n Drama\n \n Thri...,R,Paramount PicturesSee full company information...,196,Matt Damon\n \n Tom RipleySee more,Anthony Minghella\n \n Director
997,197,School of Rock (2003),"$81,261,177","$19,622,714","$35,000,000","October 3, 2003\n (Domestic)",1 hr 49 min,Comedy\n \n Music,PG-13,Paramount PicturesSee full company information...,197,Jack Black\n \n Dewey FinnSee more,Richard Linklater\n \n Director
998,198,Police Academy (1984),"$81,198,894","$8,570,007",NaN,"March 23, 1984\n (Domestic)",1 hr 36 min,Comedy,NaN,Warner Bros.See full company information\n\n,198,Steve Guttenberg\n \n Carey MahoneyS...,Hugh Wilson\n \n Director


In [170]:
top1000 = top1000.drop(columns=["index_y", "index_x"])
top1000

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
0,Star Wars: Episode VII - The Force Awakens (2015),"$936,662,225","$247,966,675","$245,000,000","December 16, 2015\n (EMEA, APAC)",2 hr 18 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Daisy Ridley\n \n ReySee more,J.J. Abrams\n \n Director
1,Avengers: Endgame (2019),"$858,373,000","$357,115,007","$356,000,000","April 24, 2019\n (21 markets)",3 hr 1 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Robert Downey Jr.\n \n Tony Stark/Ir...,Anthony Russo\n \n Director
2,Spider-Man: No Way Home (2021),"$804,761,122","$260,138,569",NaN,"December 15, 2021\n (EMEA, APAC)",2 hr 28 min,Action\n \n Adventure\n \n ...,PG-13,Sony Pictures Entertainment (SPE)See full comp...,Tom Holland,Jon Watts
3,Avatar (2009),"$760,507,625","$77,025,481","$237,000,000","December 16, 2009\n (LATAM, EMEA)",2 hr 42 min,Action\n \n Adventure\n \n ...,PG-13,Twentieth Century FoxSee full company informat...,Sam Worthington\n \n Jake SullySee more,James Cameron\n \n Director
4,Black Panther (2018),"$700,426,566","$202,003,951",NaN,"February 13, 2018\n (APAC, EMEA)",2 hr 14 min,Action\n \n Adventure\n \n ...,PG-13,Walt Disney Studios Motion PicturesSee full co...,Chadwick Boseman\n \n T'Challa/Black...,Ryan Coogler\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
995,The Notebook (2004),"$81,417,274","$13,464,745","$29,000,000","June 25, 2004\n (EMEA, Domestic)",2 hr 3 min,Drama\n \n Romance,PG-13,New Line CinemaSee full company information\n\n,Gena Rowlands\n \n Allie CalhounSee ...,Nick Cassavetes\n \n Director
996,The Talented Mr. Ripley (1999),"$81,298,265","$12,738,237","$40,000,000","December 25, 1999\n (Domestic)",2 hr 19 min,Crime\n \n Drama\n \n Thri...,R,Paramount PicturesSee full company information...,Matt Damon\n \n Tom RipleySee more,Anthony Minghella\n \n Director
997,School of Rock (2003),"$81,261,177","$19,622,714","$35,000,000","October 3, 2003\n (Domestic)",1 hr 49 min,Comedy\n \n Music,PG-13,Paramount PicturesSee full company information...,Jack Black\n \n Dewey FinnSee more,Richard Linklater\n \n Director
998,Police Academy (1984),"$81,198,894","$8,570,007",NaN,"March 23, 1984\n (Domestic)",1 hr 36 min,Comedy,NaN,Warner Bros.See full company information\n\n,Steve Guttenberg\n \n Carey MahoneyS...,Hugh Wilson\n \n Director


In [171]:
top1000.to_pickle("top1000.pkl")

In [3]:
top1000 = pd.read_pickle("top1000.pkl")  

In [9]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1000 non-null   object
 1   Domestic_Gr  1000 non-null   object
 2   Dom_Open     969 non-null    object
 3   Budget       807 non-null    object
 4   Rel_Date     1000 non-null   object
 5   Runtime      1000 non-null   object
 6   Genre        1000 non-null   object
 7   Rating       886 non-null    object
 8   Distributor  999 non-null    object
 9   Cast         1000 non-null   object
 10  Director     1000 non-null   object
dtypes: object(11)
memory usage: 126.0+ KB


In [6]:
#missing values for Rating and Budget, let's see if I can fill these in for Rating...
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
30,Toy Story 3 (2010),"$415,004,880","$110,307,189","$200,000,000","June 16, 2010\n (Egypt)",1 hr 43 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Tom Hanks\n \n WoodySee more,Lee Unkrich\n \n Director
44,Finding Nemo (2003),"$380,843,261","$70,251,710","$94,000,000","May 30, 2003\n (Domestic)",1 hr 40 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Albert Brooks\n \n MarlinSee more,Andrew Stanton\n \n Director
104,Home Alone (1990),"$285,761,243","$17,081,997","$18,000,000","November 16, 1990\n (Domestic)",1 hr 43 min,Comedy\n \n Family,NaN,Twentieth Century FoxSee full company informat...,Macaulay Culkin\n \n KevinSee more,Chris Columbus\n \n Director
118,The Incredibles (2004),"$261,441,092","$70,467,623","$92,000,000","November 5, 2004\n (APAC, Domestic)",1 hr 55 min,Action\n \n Adventure\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Craig T. Nelson\n \n Bob Parr/Mr. In...,Brad Bird\n \n Director
119,Jaws (1975),"$260,758,300","$7,061,513","$7,000,000","June 20, 1975\n (Domestic)",2 hr 4 min,Adventure\n \n Thriller,NaN,Universal PicturesSee full company information...,Roy Scheider\n \n BrodySee more,Steven Spielberg\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
970,Days of Thunder (1990),"$82,670,733","$15,490,445",NaN,"June 29, 1990\n (Domestic)",1 hr 47 min,Action\n \n Drama\n \n Sport,NaN,Paramount PicturesSee full company information...,Tom Cruise\n \n Cole TrickleSee more,Tony Scott\n \n Director
977,Fried Green Tomatoes (1991),"$82,418,501","$105,317",NaN,"December 27, 1991\n (Domestic)",2 hr 10 min,Drama,NaN,Universal PicturesSee full company information...,Kathy Bates\n \n Evelyn CouchSee more,Jon Avnet\n \n Director
986,Jaws 2 (1978),"$81,766,007","$9,866,023",NaN,"June 16, 1978\n (Domestic)",1 hr 56 min,Adventure\n \n Horror\n \n ...,NaN,Universal PicturesSee full company information...,Roy Scheider\n \n BrodySee more,Jeannot Szwarc\n \n Director
989,Heaven Can Wait (1978),"$81,640,278","$3,652,486",NaN,"June 28, 1978\n (Domestic)",1 hr 41 min,Comedy\n \n Fantasy\n \n R...,NaN,Paramount PicturesSee full company information...,Warren Beatty\n \n Joe PendletonSee ...,Warren Beatty\n \n Director


In [7]:
#missing Ratings that I need to manually add are...
top1000.loc[30, ["Rating"]] = "G"
top1000.loc[44, ["Rating"]] = "G"
top1000.loc[104, ["Rating"]] = "PG"
top1000.loc[118, ["Rating"]] = "PG"
top1000.loc[119, ["Rating"]] = "PG-13"
top1000.loc[970, ["Rating"]] = "PG-13"
top1000.loc[977, ["Rating"]] = "PG-13"
top1000.loc[986, ["Rating"]] = "PG"
top1000.loc[989, ["Rating"]] = "PG"
top1000.loc[998, ["Rating"]] = "R"

In [8]:
#sanity check to make sure all are entered right
top1000.loc[[30, 44, 104, 118, 970]]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
30,Toy Story 3 (2010),"$415,004,880","$110,307,189","$200,000,000","June 16, 2010\n (Egypt)",1 hr 43 min,Adventure\n \n Animation\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Tom Hanks\n \n WoodySee more,Lee Unkrich\n \n Director
44,Finding Nemo (2003),"$380,843,261","$70,251,710","$94,000,000","May 30, 2003\n (Domestic)",1 hr 40 min,Adventure\n \n Animation\n \n ...,G,Walt Disney Studios Motion PicturesSee full co...,Albert Brooks\n \n MarlinSee more,Andrew Stanton\n \n Director
104,Home Alone (1990),"$285,761,243","$17,081,997","$18,000,000","November 16, 1990\n (Domestic)",1 hr 43 min,Comedy\n \n Family,PG,Twentieth Century FoxSee full company informat...,Macaulay Culkin\n \n KevinSee more,Chris Columbus\n \n Director
118,The Incredibles (2004),"$261,441,092","$70,467,623","$92,000,000","November 5, 2004\n (APAC, Domestic)",1 hr 55 min,Action\n \n Adventure\n \n ...,PG,Walt Disney Studios Motion PicturesSee full co...,Craig T. Nelson\n \n Bob Parr/Mr. In...,Brad Bird\n \n Director
970,Days of Thunder (1990),"$82,670,733","$15,490,445",NaN,"June 29, 1990\n (Domestic)",1 hr 47 min,Action\n \n Drama\n \n Sport,PG-13,Paramount PicturesSee full company information...,Tom Cruise\n \n Cole TrickleSee more,Tony Scott\n \n Director


In [10]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
130,Batman (1989),"$251,409,241","$40,489,746","$35,000,000","June 23, 1989\n (Domestic)",2 hr 6 min,Action\n \n Adventure,NaN,Warner Bros.See full company information\n\n,Michael Keaton\n \n Batman/Bruce Way...,Tim Burton\n \n Director
136,Toy Story 2 (1999),"$245,852,179","$300,163","$90,000,000","November 19, 1999\n (Domestic)",1 hr 32 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Tom Hanks\n \n WoodySee more,John Lasseter\n \n Director
138,Cars (2006),"$244,082,982","$60,119,509","$120,000,000","June 8, 2006\n (APAC)",1 hr 57 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Owen Wilson\n \n Lightning McQueenSe...,John Lasseter\n \n Director
139,Ghostbusters (1984),"$243,578,797","$13,578,151","$30,000,000","June 8, 1984\n (Domestic)",1 hr 45 min,Action\n \n Comedy\n \n Fa...,NaN,Columbia PicturesSee full company information\n\n,Bill Murray\n \n Dr. Peter VenkmanSe...,Ivan Reitman\n \n Director
141,Twister (1996),"$241,830,615","$41,059,405","$92,000,000","May 10, 1996\n (Domestic)",1 hr 53 min,Action\n \n Adventure\n \n ...,NaN,Warner Bros.See full company information\n\n,Helen Hunt\n \n Dr. Jo HardingSee more,Jan de Bont\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
926,Rocky II (1979),"$85,182,160","$6,390,537",NaN,"June 15, 1979\n (Domestic)",1 hr 59 min,Drama\n \n Sport,NaN,United ArtistsSee full company information\n\n,Sylvester Stallone\n \n Rocky Balboa...,Sylvester Stallone\n \n Director
934,Steel Magnolias (1989),"$84,614,512","$5,425,440",NaN,"November 17, 1989\n (Domestic)",1 hr 57 min,Comedy\n \n Drama\n \n Rom...,NaN,TriStar PicturesSee full company information\n\n,Shirley MacLaine\n \n Ouiser Boudrea...,Herbert Ross\n \n Director
935,The Poseidon Adventure (1972),"$84,563,118","$264,850",NaN,"December 13, 1972\n (Domestic)",1 hr 57 min,Action\n \n Adventure\n \n ...,NaN,Twentieth Century FoxSee full company informat...,Gene Hackman\n \n Reverend ScottSee ...,Ronald Neame\n \n Director
949,Die Hard (1988),"$83,844,093","$601,851","$28,000,000","July 15, 1988\n (Domestic)",2 hr 12 min,Action\n \n Thriller,NaN,Twentieth Century FoxSee full company informat...,Bruce Willis\n \n John McClaneSee more,John McTiernan\n \n Director


In [11]:
#missing Ratings that I need to manually add are...
top1000.loc[130, ["Rating"]] = "PG-13"
top1000.loc[136, ["Rating"]] = "G"
top1000.loc[138, ["Rating"]] = "G"
top1000.loc[139, ["Rating"]] = "PG"
top1000.loc[141, ["Rating"]] = "PG-13"
top1000.loc[926, ["Rating"]] = "PG"
top1000.loc[934, ["Rating"]] = "PG"
top1000.loc[935, ["Rating"]] = "PG"
top1000.loc[949, ["Rating"]] = "R"
top1000.loc[958, ["Rating"]] = "PG"

In [12]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
149,Beverly Hills Cop (1984),"$234,760,478","$15,214,805",NaN,"December 5, 1984\n (Domestic)",1 hr 45 min,Action\n \n Comedy\n \n Cr...,NaN,Paramount PicturesSee full company information...,Eddie Murphy\n \n Axel FoleySee more,Martin Brest\n \n Director
169,WALL·E (2008),"$223,808,164","$63,087,526","$180,000,000","June 26, 2008\n (LATAM)",1 hr 38 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Ben Burtt\n \n WALL·E/M-O/RobotsSee ...,Andrew Stanton\n \n Director
170,Toy Story (1995),"$223,225,679","$29,140,617",NaN,"November 22, 1995\n (Domestic)",1 hr 21 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Tom Hanks\n \n WoodySee more,John Lasseter\n \n Director
176,Beauty and the Beast (1991),"$218,967,620","$162,146","$25,000,000","November 15, 1991\n (Domestic)",1 hr 24 min,Animation\n \n Family\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Paige O'Hara\n \n BelleSee more,Gary Trousdale\n \n Director
179,Ghost (1990),"$217,631,306","$12,191,540","$22,000,000","July 13, 1990\n (Domestic)",2 hr 7 min,Drama\n \n Fantasy\n \n Ro...,NaN,Paramount PicturesSee full company information...,Patrick Swayze\n \n Sam WheatSee more,Jerry Zucker\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
906,Presumed Innocent (1990),"$86,303,188","$11,718,981",NaN,"July 27, 1990\n (Domestic)",2 hr 7 min,Mystery\n \n Thriller,NaN,Warner Bros.See full company information\n\n,Harrison Ford\n \n Rusty SabichSee more,Alan J. Pakula\n \n Director
907,Young Frankenstein (1974),"$86,273,333",NaN,"$2,800,000","December 15, 1974\n (Domestic)",1 hr 46 min,Comedy,NaN,Twentieth Century FoxSee full company informat...,Gene Wilder\n \n Dr. Frederick Frank...,Mel Brooks\n \n Director
918,Pokémon: The First Movie - Mewtwo Strikes Back...,"$85,744,662","$31,036,678","$30,000,000","November 10, 1999\n (Domestic)",1 hr 36 min,Action\n \n Adventure\n \n ...,NaN,Warner Bros.See full company information\n\n,Veronica Taylor\n \n Ash Ketchum/Fem...,Kunihiko Yuyama\n \n Director
922,Brother Bear (2003),"$85,336,277","$291,940",NaN,"October 24, 2003\n (Domestic)",1 hr 25 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Joaquin Phoenix\n \n KenaiSee more,Aaron Blaise\n \n Director


In [13]:
#missing Ratings that I need to manually add are...
top1000.loc[149, ["Rating"]] = "R"
top1000.loc[169, ["Rating"]] = "G"
top1000.loc[170, ["Rating"]] = "G"
top1000.loc[176, ["Rating"]] = "PG"
top1000.loc[179, ["Rating"]] = "PG-13"
top1000.loc[906, ["Rating"]] = "R"
top1000.loc[907, ["Rating"]] = "PG"
top1000.loc[918, ["Rating"]] = "G"
top1000.loc[922, ["Rating"]] = "G"
top1000.loc[925, ["Rating"]] = "PG"

In [14]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
182,Aladdin (1992),"$217,350,219","$196,664","$28,000,000","November 13, 1992\n (Domestic)",1 hr 30 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Scott Weinger\n \n AladdinSee more,Ron Clements\n \n Director
197,Back to the Future (1985),"$212,836,762","$11,152,500","$19,000,000","July 3, 1985\n (Domestic)",1 hr 56 min,Adventure\n \n Comedy\n \n ...,NaN,Universal PicturesSee full company information...,Michael J. Fox\n \n Marty McFlySee more,Robert Zemeckis\n \n Director
219,Gone with the Wind (1939),"$200,882,193",NaN,NaN,"December 15, 1939\n (Domestic)",3 hr 58 min,Drama\n \n Romance\n \n War,NaN,Metro-Goldwyn-Mayer (MGM)See full company info...,Clark Gable\n \n Rhett Butler - Visi...,Victor Fleming\n \n Director
241,The Polar Express (2004),"$188,969,804","$23,323,463","$165,000,000","November 10, 2004\n (Domestic)",1 hr 40 min,Adventure\n \n Animation\n \n ...,NaN,Warner Bros.See full company information\n\n,Tom Hanks\n \n Hero Boy/Father/Condu...,Robert Zemeckis\n \n Director
252,Dances with Wolves (1990),"$184,208,848","$598,257","$22,000,000","November 9, 1990\n (Domestic)",3 hr 1 min,Adventure\n \n Drama\n \n ...,NaN,Orion PicturesSee full company information\n\n,Kevin Costner\n \n Lieutenant Dunbar...,Kevin Costner\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
898,Everest (1998),"$87,178,599","$364,244",NaN,"March 6, 1998\n (Domestic)",44 min,Documentary\n \n Short,NaN,MacGillivray Freeman FilmsSee full company inf...,Liam Neeson\n \n Self - NarratorSee ...,David Breashears\n \n Director
899,Out of Africa (1985),"$87,071,205","$3,637,290",NaN,"December 20, 1985\n (Domestic)",2 hr 41 min,Biography\n \n Drama\n \n ...,NaN,Universal PicturesSee full company information...,Meryl Streep\n \n KarenSee more,Sydney Pollack\n \n Director
902,The Naked Gun 2½: The Smell of Fear (1991),"$86,930,411","$20,817,139","$23,000,000","June 28, 1991\n (Domestic)",1 hr 25 min,Comedy\n \n Crime,NaN,Paramount PicturesSee full company information...,Leslie Nielsen\n \n Lt. Frank Drebin...,David Zucker\n \n Director
904,The War of the Roses (1989),"$86,888,546","$9,488,794",NaN,"December 8, 1989\n (Domestic)",1 hr 56 min,Comedy\n \n Romance,NaN,Twentieth Century FoxSee full company informat...,Michael Douglas\n \n Oliver RoseSee ...,Danny DeVito\n \n Director


In [15]:
#missing Ratings that I need to manually add are...
top1000.loc[182, ["Rating"]] = "G"
top1000.loc[197, ["Rating"]] = "PG"
top1000.loc[219, ["Rating"]] = "G"
top1000.loc[241, ["Rating"]] = "G"
top1000.loc[252, ["Rating"]] = "PG-13"
top1000.loc[898, ["Rating"]] = "PG-13"
top1000.loc[899, ["Rating"]] = "PG"
top1000.loc[902, ["Rating"]] = "PG-13"
top1000.loc[904, ["Rating"]] = "R"
top1000.loc[905, ["Rating"]] = "R"

In [16]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
266,Top Gun (1986),"$180,258,178","$8,193,052",NaN,"May 16, 1986\n (Domestic)",1 hr 50 min,Action\n \n Drama,NaN,Paramount PicturesSee full company information...,Tom Cruise\n \n MaverickSee more,Tony Scott\n \n Director
271,Indiana Jones and the Temple of Doom (1984),"$179,870,271","$25,337,110","$28,000,000","May 23, 1984\n (Domestic)",1 hr 58 min,Action\n \n Adventure,NaN,Paramount PicturesSee full company information...,Harrison Ford\n \n Indiana JonesSee ...,Steven Spielberg\n \n Director
278,Tootsie (1982),"$177,200,000","$5,540,470","$21,000,000","December 17, 1982\n (Domestic)",1 hr 56 min,Comedy\n \n Drama\n \n Rom...,NaN,Columbia PicturesSee full company information\n\n,Dustin Hoffman\n \n Michael Dorsey/D...,Sydney Pollack\n \n Director
291,Crocodile Dundee (1986),"$174,803,506","$8,038,855",NaN,"September 26, 1986\n (Domestic)",1 hr 37 min,Action\n \n Adventure\n \n ...,NaN,Paramount PicturesSee full company information...,Paul Hogan\n \n Mick 'Crocodile' Dun...,Peter Faiman\n \n Director
293,Fast & Furious Presents: Hobbs & Shaw (2019),"$173,956,935","$60,038,950","$200,000,000","July 1, 2019\n (Portugal)",2 hr 17 min,Action\n \n Adventure\n \n ...,NaN,Universal PicturesSee full company information...,Dwayne Johnson\n \n HobbsSee more,David Leitch\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
857,The Karate Kid (1984),"$91,077,276","$5,031,753",NaN,"June 22, 1984\n (Domestic)",2 hr 6 min,Action\n \n Drama\n \n Fam...,NaN,Columbia PicturesSee full company information\n\n,Ralph Macchio\n \n DanielSee more,John G. Avildsen\n \n Director
870,Trading Places (1983),"$90,404,800","$7,348,200",NaN,"June 10, 1983\n (Domestic)",1 hr 56 min,Comedy,NaN,Paramount PicturesSee full company information...,Eddie Murphy\n \n Billy Ray Valentin...,John Landis\n \n Director
877,The Emperor's New Groove (2000),"$89,636,687","$9,812,302","$100,000,000","December 15, 2000\n (Domestic)",1 hr 18 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,David Spade\n \n KuzcoSee more,Mark Dindal\n \n Director
878,Father of the Bride (1991),"$89,325,780","$7,031,146",NaN,"December 20, 1991\n (Domestic)",1 hr 45 min,Comedy\n \n Family\n \n Ro...,NaN,Walt Disney Studios Motion PicturesSee full co...,Steve Martin\n \n George BanksSee more,Charles Shyer\n \n Director


In [17]:
#missing Ratings that I need to manually add are...
top1000.loc[266, ["Rating"]] = "PG"
top1000.loc[271, ["Rating"]] = "PG"
top1000.loc[278, ["Rating"]] = "PG"
top1000.loc[291, ["Rating"]] = "PG-13"
top1000.loc[293, ["Rating"]] = "PG-13"
top1000.loc[857, ["Rating"]] = "PG"
top1000.loc[870, ["Rating"]] = "R"
top1000.loc[877, ["Rating"]] = "G"
top1000.loc[878, ["Rating"]] = "PG-13"
top1000.loc[889, ["Rating"]] = "PG"

In [18]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
299,Rain Man (1988),"$172,825,435","$7,005,719","$25,000,000","December 16, 1988\n (Domestic)",2 hr 13 min,Drama,NaN,Metro-Goldwyn-Mayer (MGM)See full company info...,Dustin Hoffman\n \n Raymond BabbittS...,Barry Levinson\n \n Director
316,Three Men and a Baby (1987),"$167,780,960","$10,384,392","$11,000,000","November 25, 1987\n (Domestic)",1 hr 42 min,Comedy\n \n Drama\n \n Family,NaN,Walt Disney Studios Motion PicturesSee full co...,Tom Selleck\n \n PeterSee more,Leonard Nimoy\n \n Director
323,Robin Hood: Prince of Thieves (1991),"$165,493,908","$25,625,602","$48,000,000","June 14, 1991\n (Domestic)",2 hr 23 min,Action\n \n Adventure\n \n ...,NaN,Warner Bros.See full company information\n\n,Kevin Costner\n \n Robin of Locksley...,Kevin Reynolds\n \n Director
336,A Bug's Life (1998),"$162,798,565","$291,121","$120,000,000","November 20, 1998\n (Domestic)",1 hr 35 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Kevin Spacey\n \n HopperSee more,John Lasseter\n \n Director
351,The Sound of Music (1965),"$159,287,539",NaN,"$8,200,000","March 2, 1965\n (Domestic)",2 hr 52 min,Biography\n \n Drama\n \n ...,NaN,Twentieth Century FoxSee full company informat...,Julie Andrews\n \n MariaSee more,Robert Wise\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
833,Space Station 3D (2002),"$93,383,953","$489,488",NaN,"April 19, 2002\n (Domestic)",47 min,Documentary,NaN,IMAXSee full company information\n\n,Tom Cruise\n \n NarratorSee more,Toni Myers\n \n Director
837,When Harry Met Sally... (1989),"$93,117,425","$1,094,453",NaN,"July 14, 1989\n (Domestic)",1 hr 35 min,Comedy\n \n Drama\n \n Rom...,NaN,Columbia PicturesSee full company information\n\n,Billy Crystal\n \n Harry BurnsSee more,Rob Reiner\n \n Director
840,Flashdance (1983),"$92,921,203","$4,076,124",NaN,"April 15, 1983\n (Domestic)",1 hr 35 min,Drama\n \n Music\n \n Romance,NaN,Paramount PicturesSee full company information...,Jennifer Beals\n \n Alex OwensSee more,Adrian Lyne\n \n Director
849,Kindergarten Cop (1990),"$91,457,688","$7,918,560","$15,000,000","December 22, 1990\n (Domestic)",1 hr 51 min,Action\n \n Comedy\n \n Crime,NaN,Universal PicturesSee full company information...,Arnold Schwarzenegger\n \n KimbleSee...,Ivan Reitman\n \n Director


In [19]:
#missing Ratings that I need to manually add are...
top1000.loc[299, ["Rating"]] = "R"
top1000.loc[316, ["Rating"]] = "PG"
top1000.loc[323, ["Rating"]] = "PG-13"
top1000.loc[336, ["Rating"]] = "G"
top1000.loc[351, ["Rating"]] = "G"
top1000.loc[833, ["Rating"]] = "R"
top1000.loc[837, ["Rating"]] = "R"
top1000.loc[840, ["Rating"]] = "R"
top1000.loc[849, ["Rating"]] = "PG-13"
top1000.loc[853, ["Rating"]] = "PG-13"

In [20]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
359,Fatal Attraction (1987),"$156,645,693","$7,602,740",NaN,"September 18, 1987\n (Domestic)",1 hr 59 min,Drama\n \n Thriller,NaN,Paramount PicturesSee full company information...,Michael Douglas\n \n Dan GallagherSe...,Adrian Lyne\n \n Director
360,Who Framed Roger Rabbit (1988),"$156,452,370","$11,226,239","$70,000,000","June 24, 1988\n (Domestic)",1 hr 44 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Bob Hoskins\n \n Eddie ValiantSee more,Robert Zemeckis\n \n Director
361,The Sting (1973),"$156,000,000",NaN,NaN,"December 25, 1973\n (Domestic)",2 hr 9 min,Comedy\n \n Crime\n \n Drama,NaN,Universal PicturesSee full company information...,Paul Newman\n \n Henry GondorffSee more,George Roy Hill\n \n Director
370,Horton Hears a Who! (2008),"$154,529,439","$45,012,998","$85,000,000","March 12, 2008\n (LATAM)",1 hr 26 min,Adventure\n \n Animation\n \n ...,NaN,Twentieth Century FoxSee full company informat...,Jim Carrey\n \n HortonSee more,Jimmy Hayward\n \n Director
374,Beverly Hills Cop II (1987),"$153,665,036","$26,348,555",NaN,"May 20, 1987\n (Domestic)",1 hr 40 min,Action\n \n Comedy\n \n Cr...,NaN,Paramount PicturesSee full company information...,Eddie Murphy\n \n Axel FoleySee more,Tony Scott\n \n Director
...,...,...,...,...,...,...,...,...,...,...,...
791,The Color Purple (1985),"$98,467,863","$1,710,333",NaN,"December 20, 1985\n (Domestic)",2 hr 34 min,Drama,NaN,Warner Bros.See full company information\n\n,Danny Glover\n \n AlbertSee more,Steven Spielberg\n \n Director
801,Legally Blonde (2001),"$96,520,674","$20,377,426","$18,000,000","July 13, 2001\n (Domestic)",1 hr 36 min,Comedy\n \n Romance,NaN,Metro-Goldwyn-Mayer (MGM)See full company info...,Reese Witherspoon\n \n Elle WoodsSee...,Robert Luketic\n \n Director
805,Dead Poets Society (1989),"$95,860,116","$340,456",NaN,"June 2, 1989\n (Domestic)",2 hr 8 min,Comedy\n \n Drama,NaN,Walt Disney Studios Motion PicturesSee full co...,Robin Williams\n \n John KeatingSee ...,Peter Weir\n \n Director
809,Arthur (1981),"$95,461,682","$2,719,534",NaN,"July 17, 1981\n (Domestic)",1 hr 37 min,Comedy\n \n Romance,NaN,Warner Bros.See full company information\n\n,Dudley Moore\n \n Arthur BachSee more,Steve Gordon\n \n Director


In [21]:
#missing Ratings that I need to manually add are...
top1000.loc[359, ["Rating"]] = "R"
top1000.loc[360, ["Rating"]] = "PG"
top1000.loc[361, ["Rating"]] = "PG"
top1000.loc[370, ["Rating"]] = "G"
top1000.loc[374, ["Rating"]] = "R"
top1000.loc[791, ["Rating"]] = "PG-13"
top1000.loc[801, ["Rating"]] = "PG-13"
top1000.loc[805, ["Rating"]] = "PG"
top1000.loc[809, ["Rating"]] = "PG"
top1000.loc[813, ["Rating"]] = "G"

In [22]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1000 non-null   object
 1   Domestic_Gr  1000 non-null   object
 2   Dom_Open     969 non-null    object
 3   Budget       807 non-null    object
 4   Rel_Date     1000 non-null   object
 5   Runtime      1000 non-null   object
 6   Genre        1000 non-null   object
 7   Rating       946 non-null    object
 8   Distributor  999 non-null    object
 9   Cast         1000 non-null   object
 10  Director     1000 non-null   object
dtypes: object(11)
memory usage: 126.0+ KB


In [23]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
375,Gremlins (1984),"$153,642,180","$12,511,634",NaN,"June 8, 1984\n (Domestic)",1 hr 46 min,Comedy\n \n Fantasy\n \n H...,NaN,Warner Bros.See full company information\n\n,Zach Galligan\n \n BillySee more,Joe Dante\n \n Director
382,Rambo: First Blood Part II (1985),"$150,415,432","$20,176,217",NaN,"May 22, 1985\n (Domestic)",1 hr 36 min,Action\n \n Adventure\n \n ...,NaN,TriStar PicturesSee full company information\n\n,Sylvester Stallone\n \n RamboSee more,George P. Cosmatos\n \n Director
410,One Hundred and One Dalmatians (1961),"$144,880,014",NaN,NaN,"January 25, 1961\n (Domestic)",1 hr 19 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Rod Taylor\n \n PongoSee more,Clyde Geronimi\n \n Director
427,National Lampoon's Animal House (1978),"$141,600,000","$276,538",NaN,"July 28, 1978\n (Domestic)",1 hr 49 min,Comedy,NaN,Universal PicturesSee full company information...,John Belushi\n \n John BlutarskySee ...,John Landis\n \n Director
437,Look Who's Talking (1989),"$140,088,813","$12,107,784",NaN,"October 13, 1989\n (Domestic)",1 hr 33 min,Comedy,NaN,TriStar PicturesSee full company information\n\n,John Travolta\n \n JamesSee more,Amy Heckerling\n \n Director
442,The Santa Clause 2 (2002),"$139,236,327","$29,008,696","$65,000,000","November 1, 2002\n (Domestic)",1 hr 44 min,Comedy\n \n Family\n \n Fa...,NaN,Walt Disney Studios Motion PicturesSee full co...,Tim Allen\n \n Scott Calvin/Santa/To...,Michael Lembeck\n \n Director
446,Platoon (1986),"$138,530,565","$241,080","$6,000,000","December 19, 1986\n (Domestic)",2 hr,Drama\n \n War,NaN,Orion PicturesSee full company information\n\n,Charlie Sheen\n \n ChrisSee more,Oliver Stone\n \n Director
457,The Godfather (1972),"$136,381,073","$302,393","$6,000,000","March 15, 1972\n (Domestic)",2 hr 55 min,Crime\n \n Drama,NaN,Paramount PicturesSee full company information...,Marlon Brando\n \n Don Vito Corleone...,Francis Ford Coppola\n \n Director
459,101 Dalmatians (1996),"$136,189,294","$33,504,025",NaN,"November 27, 1996\n (Domestic)",1 hr 43 min,Adventure\n \n Comedy\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Glenn Close\n \n Cruella DeVilSee more,Stephen Herek\n \n Director
463,Teenage Mutant Ninja Turtles (1990),"$135,384,756","$25,398,367","$13,500,000","March 30, 1990\n (Domestic)",1 hr 33 min,Action\n \n Adventure\n \n ...,NaN,New Line CinemaSee full company information\n\n,Judith Hoag\n \n April O'NeilSee more,Steve Barron\n \n Director


In [24]:
#missing Ratings that I need to manually add are...
top1000.loc[375, ["Rating"]] = "PG"
top1000.loc[382, ["Rating"]] = "PG-13"
top1000.loc[410, ["Rating"]] = "G"
top1000.loc[427, ["Rating"]] = "R"
top1000.loc[437, ["Rating"]] = "PG-13"
top1000.loc[442, ["Rating"]] = "G"
top1000.loc[446, ["Rating"]] = "R"
top1000.loc[457, ["Rating"]] = "R"
top1000.loc[459, ["Rating"]] = "G"
top1000.loc[463, ["Rating"]] = "PG"


In [25]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
464,Close Encounters of the Third Kind (1977),"$135,189,114",NaN,"$20,000,000","November 16, 1977\n (Domestic)",2 hr 18 min,Drama\n \n Sci-Fi,NaN,Columbia PicturesSee full company information\n\n,Richard Dreyfuss\n \n Roy NearySee more,Steven Spielberg\n \n Director
489,The Silence of the Lambs (1991),"$130,742,922","$13,766,814","$19,000,000","February 14, 1991\n (Domestic)",1 hr 58 min,Crime\n \n Drama\n \n Thri...,NaN,Orion PicturesSee full company information\n\n,Jodie Foster\n \n Clarice StarlingSe...,Jonathan Demme\n \n Director
490,"Honey, I Shrunk the Kids (1989)","$130,724,172","$14,262,961",NaN,"June 23, 1989\n (Domestic)",1 hr 33 min,Adventure\n \n Comedy\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Rick Moranis\n \n Wayne SzalinskiSee...,Joe Johnston\n \n Director
499,An Officer and a Gentleman (1982),"$129,795,554","$3,304,679",NaN,"July 30, 1982\n (Domestic)",2 hr 4 min,Drama\n \n Romance,NaN,Paramount PicturesSee full company information...,Richard Gere\n \n Zack MayoSee more,Taylor Hackford\n \n Director
507,Coming to America (1988),"$128,152,301","$21,404,420",NaN,"June 29, 1988\n (Domestic)",1 hr 57 min,Comedy\n \n Romance,NaN,Paramount PicturesSee full company information...,Eddie Murphy\n \n Prince Akeem/Clare...,John Landis\n \n Director
518,Dumb and Dumber (1994),"$127,190,327","$16,363,442","$17,000,000","December 16, 1994\n (Domestic)",1 hr 47 min,Comedy,NaN,New Line CinemaSee full company information\n\n,Jim Carrey\n \n LloydSee more,Peter Farrelly\n \n Director
524,Smokey and the Bandit (1977),"$126,737,428",NaN,NaN,"May 27, 1977\n (Domestic)",1 hr 36 min,Action\n \n Adventure\n \n ...,NaN,Universal PicturesSee full company information...,Burt Reynolds\n \n BanditSee more,Hal Needham\n \n Director
539,Rocky III (1982),"$125,049,125","$12,431,486",NaN,"May 28, 1982\n (Domestic)",1 hr 39 min,Drama\n \n Sport,NaN,United ArtistsSee full company information\n\n,Sylvester Stallone\n \n Rocky Balboa...,Sylvester Stallone\n \n Director
548,City Slickers (1991),"$124,033,791","$13,032,121","$27,000,000","June 7, 1991\n (Domestic)",1 hr 53 min,Comedy\n \n Western,NaN,Columbia PicturesSee full company information\n\n,Billy Crystal\n \n Mitch RobbinsSee ...,Ron Underwood\n \n Director
549,"Good Morning, Vietnam (1987)","$123,922,370","$194,308",NaN,"December 25, 1987\n (Domestic)",2 hr 1 min,Biography\n \n Comedy\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Robin Williams\n \n Adrian CronauerS...,Barry Levinson\n \n Director


In [26]:

top1000.loc[464, ["Rating"]] = "PG"
top1000.loc[489, ["Rating"]] = "PG-13"
top1000.loc[490, ["Rating"]] = "PG"
top1000.loc[499, ["Rating"]] = "R"
top1000.loc[507, ["Rating"]] = "R"
top1000.loc[518, ["Rating"]] = "PG-13"
top1000.loc[524, ["Rating"]] = "PG"
top1000.loc[539, ["Rating"]] = "PG"
top1000.loc[548, ["Rating"]] = "PG-13"
top1000.loc[549, ["Rating"]] = "R"



In [27]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series]

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
556,The Hunt for Red October (1990),"$122,012,643","$17,161,835","$30,000,000","March 2, 1990\n (Domestic)",2 hr 15 min,Action\n \n Adventure\n \n ...,NaN,Paramount PicturesSee full company information...,Sean Connery\n \n Marko RamiusSee more,John McTiernan\n \n Director
564,Mulan (1998),"$120,620,254","$22,745,143",NaN,"June 19, 1998\n (Domestic)",1 hr 28 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Ming-Na Wen\n \n MulanSee more,Tony Bancroft\n \n Director
572,Hook (1991),"$119,654,823","$13,522,535","$70,000,000","December 11, 1991\n (Domestic)",2 hr 22 min,Adventure\n \n Comedy\n \n ...,NaN,TriStar PicturesSee full company information\n\n,Dustin Hoffman\n \n Captain HookSee ...,Steven Spielberg\n \n Director
573,Blazing Saddles (1974),"$119,616,663",NaN,NaN,"February 7, 1974\n (Domestic)",1 hr 33 min,Comedy\n \n Western,NaN,Warner Bros.See full company information\n\n,Cleavon Little\n \n BartSee more,Mel Brooks\n \n Director
576,Total Recall (1990),"$119,412,921","$25,533,700","$65,000,000","June 1, 1990\n (Domestic)",1 hr 53 min,Action\n \n Adventure\n \n ...,NaN,Sony Pictures Entertainment (SPE)See full comp...,Arnold Schwarzenegger\n \n QuaidSee ...,Paul Verhoeven\n \n Director
577,On Golden Pond (1981),"$119,285,432","$89,213",NaN,"December 4, 1981\n (Domestic)",1 hr 49 min,Drama,NaN,Universal PicturesSee full company information...,Katharine Hepburn\n \n Ethel ThayerS...,Mark Rydell\n \n Director
580,Back to the Future Part II (1989),"$119,000,002","$27,835,125","$40,000,000","November 22, 1989\n (Domestic)",1 hr 48 min,Adventure\n \n Comedy\n \n ...,NaN,Universal PicturesSee full company information...,Michael J. Fox\n \n Marty McFly/Mart...,Robert Zemeckis\n \n Director
591,Die Hard 2 (1990),"$117,540,947","$21,744,661","$70,000,000","July 6, 1990\n (Domestic)",2 hr 4 min,Action\n \n Thriller,NaN,Twentieth Century FoxSee full company informat...,Bruce Willis\n \n John McClaneSee more,Renny Harlin\n \n Director
594,Rocky (1976),"$117,235,147",NaN,NaN,"November 21, 1976\n (Domestic)",2 hr,Drama\n \n Sport,NaN,United ArtistsSee full company information\n\n,Sylvester Stallone\n \n RockySee more,John G. Avildsen\n \n Director
605,The Towering Inferno (1974),"$116,000,000",NaN,"$14,000,000","December 14, 1974\n (Domestic)",2 hr 45 min,Action\n \n Drama\n \n Thr...,NaN,Twentieth Century FoxSee full company informat...,Paul Newman\n \n Doug RobertsSee more,John Guillermin\n \n Director


In [28]:
top1000.loc[556, ["Rating"]] = "PG"
top1000.loc[564, ["Rating"]] = "G"
top1000.loc[572, ["Rating"]] = "PG"
top1000.loc[573, ["Rating"]] = "R"
top1000.loc[576, ["Rating"]] = "R"
top1000.loc[577, ["Rating"]] = "PG"
top1000.loc[580, ["Rating"]] = "PG"
top1000.loc[591, ["Rating"]] = "R"
top1000.loc[594, ["Rating"]] = "PG"
top1000.loc[605, ["Rating"]] = "PG"


In [30]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
613,Big (1988),"$115,227,281","$8,216,190",NaN,"June 3, 1988\n (Domestic)",1 hr 44 min,Comedy\n \n Drama\n \n Fan...,NaN,Twentieth Century FoxSee full company informat...,Tom Hanks\n \n JoshSee more,Penny Marshall\n \n Director
615,The Karate Kid Part II (1986),"$115,103,979","$12,652,336",NaN,"June 20, 1986\n (Domestic)",1 hr 53 min,Action\n \n Family\n \n Sport,NaN,Columbia PicturesSee full company information\n\n,Pat Morita\n \n MiyagiSee more,John G. Avildsen\n \n Director
616,American Graffiti (1973),"$115,000,000",NaN,NaN,"August 1, 1973\n (Domestic)",1 hr 50 min,Comedy\n \n Drama,NaN,Universal PicturesSee full company information...,Richard Dreyfuss\n \n CurtSee more,George Lucas\n \n Director
626,The Addams Family (1991),"$113,502,426","$24,203,754","$30,000,000","November 22, 1991\n (Domestic)",1 hr 39 min,Comedy\n \n Fantasy,NaN,Paramount PicturesSee full company information...,Anjelica Huston\n \n Morticia Addams...,Barry Sonnenfeld\n \n Director
632,The Rocky Horror Picture Show (1975),"$112,892,319",NaN,NaN,"September 26, 1975\n (Domestic)",1 hr 40 min,Comedy\n \n Horror\n \n Mu...,NaN,Twentieth Century FoxSee full company informat...,Tim Curry\n \n Dr. Frank-N-Furter - ...,Jim Sharman\n \n Director


In [31]:
top1000.loc[613, ["Rating"]] = "PG"
top1000.loc[615, ["Rating"]] = "PG"
top1000.loc[616, ["Rating"]] = "PG"
top1000.loc[626, ["Rating"]] = "PG-13"
top1000.loc[632, ["Rating"]] = "R"

In [32]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
635,Ghostbusters II (1989),"$112,494,738","$29,472,894",NaN,"June 16, 1989\n (Domestic)",1 hr 48 min,Action\n \n Comedy\n \n Fa...,NaN,Columbia PicturesSee full company information\n\n,Bill Murray\n \n Dr. Peter VenkmanSe...,Ivan Reitman\n \n Director
638,Twins (1988),"$111,938,388","$11,174,980","$15,000,000","December 9, 1988\n (Domestic)",1 hr 47 min,Comedy\n \n Crime,NaN,Universal PicturesSee full company information...,Arnold Schwarzenegger\n \n Julius Be...,Ivan Reitman\n \n Director
642,The Little Mermaid (1989),"$111,543,479","$6,031,914",NaN,"November 17, 1989\n (Domestic)",1 hr 23 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Jodi Benson\n \n ArielSee more,Ron Clements\n \n Director
644,Porky's (1981),"$111,289,673","$7,623,988",NaN,"March 19, 1982\n (Domestic)",1 hr 34 min,Comedy,NaN,Twentieth Century FoxSee full company informat...,Dan Monahan\n \n Pee WeeSee more,Bob Clark\n \n Director
660,Crocodile Dundee II (1988),"$109,306,210","$24,462,976",NaN,"May 25, 1988\n (Domestic)",1 hr 48 min,Action\n \n Adventure\n \n ...,NaN,Paramount PicturesSee full company information...,Paul Hogan\n \n Mick 'Crocodile' Dun...,John Cornell\n \n Director


In [33]:
top1000.loc[635, ["Rating"]] = "PG"
top1000.loc[638, ["Rating"]] = "PG"
top1000.loc[642, ["Rating"]] = "G"
top1000.loc[644, ["Rating"]] = "R"
top1000.loc[660, ["Rating"]] = "PG"

In [34]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
662,One Flew Over the Cuckoo's Nest (1975),"$108,981,275",NaN,"$3,000,000","November 20, 1975\n (Domestic)",2 hr 13 min,Drama,NaN,United ArtistsSee full company information\n\n,Jack Nicholson\n \n R.P. McMurphySee...,Milos Forman\n \n Director
666,Terms of Endearment (1983),"$108,423,489","$3,498,813","$8,000,000","November 23, 1983\n (Domestic)",2 hr 12 min,Comedy\n \n Drama,NaN,Paramount PicturesSee full company information...,Shirley MacLaine\n \n Aurora Greenwa...,James L. Brooks\n \n Director
684,Chicken Run (2000),"$106,834,564","$17,506,162","$45,000,000","June 23, 2000\n (Domestic)",1 hr 24 min,Adventure\n \n Animation\n \n ...,NaN,DreamWorks DistributionSee full company inform...,Mel Gibson\n \n RockySee more,Peter Lord\n \n Director
687,Driving Miss Daisy (1989),"$106,593,296","$73,745",NaN,"December 15, 1989\n (Domestic)",1 hr 39 min,Comedy\n \n Drama\n \n Rom...,NaN,Warner Bros.See full company information\n\n,Morgan Freeman\n \n Hoke ColburnSee ...,Bruce Beresford\n \n Director
705,The Princess and the Frog (2009),"$104,400,899","$786,190","$105,000,000","November 25, 2009\n (Domestic)",1 hr 37 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Anika Noni Rose\n \n TianaSee more,Ron Clements\n \n Director


In [35]:
top1000.loc[662, ["Rating"]] = "R"
top1000.loc[666, ["Rating"]] = "PG"
top1000.loc[684, ["Rating"]] = "G"
top1000.loc[687, ["Rating"]] = "PG"
top1000.loc[705, ["Rating"]] = "G"

In [36]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
712,Dick Tracy (1990),"$103,738,726","$22,543,911","$47,000,000","June 15, 1990\n (Domestic)",1 hr 45 min,Action\n \n Comedy\n \n Cr...,NaN,Walt Disney Studios Motion PicturesSee full co...,Warren Beatty\n \n Dick TracySee more,Warren Beatty\n \n Director
715,9 to 5 (1980),"$103,290,500","$3,966,832",NaN,"December 19, 1980\n (Domestic)",1 hr 49 min,Comedy,NaN,Twentieth Century FoxSee full company informat...,Jane Fonda\n \n Judy BernlySee more,Colin Higgins\n \n Director
747,Stir Crazy (1980),"$101,300,000","$8,691,886",NaN,"December 12, 1980\n (Domestic)",1 hr 51 min,Comedy\n \n Crime,NaN,Columbia PicturesSee full company information\n\n,Gene Wilder\n \n Skip DonahueSee more,Sidney Poitier\n \n Director
765,The Rugrats Movie (1998),"$100,494,675","$27,321,470","$24,000,000","November 20, 1998\n (Domestic)",1 hr 19 min,Adventure\n \n Animation\n \n ...,NaN,Paramount PicturesSee full company information...,Elizabeth Daily\n \n Tommy PicklesSe...,Igor Kovalyov\n \n Director
767,Airport (1970),"$100,489,151",NaN,"$10,000,000","March 5, 1970\n (Domestic)",2 hr 17 min,Action\n \n Drama\n \n Thr...,NaN,Universal PicturesSee full company information...,Burt Lancaster\n \n Mel BakersfeldSe...,George Seaton\n \n Director


In [37]:
top1000.loc[712, ["Rating"]] = "PG"
top1000.loc[715, ["Rating"]] = "PG"
top1000.loc[747, ["Rating"]] = "R"
top1000.loc[765, ["Rating"]] = "G"
top1000.loc[767, ["Rating"]] = "G"

In [38]:
#continuing on....
bool_series = pd.isnull(top1000["Rating"])
top1000[bool_series].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
779,The Hunchback of Notre Dame (1996),"$100,138,851","$21,037,414","$100,000,000","June 21, 1996\n (Domestic)",1 hr 31 min,Animation\n \n Drama\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Demi Moore\n \n EsmeraldaSee more,Gary Trousdale\n \n Director
781,Parenthood (1989),"$100,047,830","$10,506,450",NaN,"August 2, 1989\n (Domestic)",2 hr 4 min,Comedy\n \n Drama,NaN,Universal PicturesSee full company information...,Steve Martin\n \n GilSee more,Ron Howard\n \n Director
785,Gnomeo & Juliet (2011),"$99,967,670","$25,356,909",NaN,"February 10, 2011\n (APAC, EMEA)",1 hr 24 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,James McAvoy\n \n GnomeoSee more,Kelly Asbury\n \n Director
787,Hercules (1997),"$99,112,101","$249,567","$85,000,000","June 15, 1997\n (Domestic)",1 hr 33 min,Adventure\n \n Animation\n \n ...,NaN,Walt Disney Studios Motion PicturesSee full co...,Tate Donovan\n \n HerculesSee more,Ron Clements\n \n Director


In [39]:
top1000.loc[779, ["Rating"]] = "G"
top1000.loc[781, ["Rating"]] = "PG-13"
top1000.loc[785, ["Rating"]] = "G"
top1000.loc[787, ["Rating"]] = "G"

In [41]:
top1000.info()
#ALL BUDGET filled in now!!!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1000 non-null   object
 1   Domestic_Gr  1000 non-null   object
 2   Dom_Open     969 non-null    object
 3   Budget       807 non-null    object
 4   Rel_Date     1000 non-null   object
 5   Runtime      1000 non-null   object
 6   Genre        1000 non-null   object
 7   Rating       1000 non-null   object
 8   Distributor  999 non-null    object
 9   Cast         1000 non-null   object
 10  Director     1000 non-null   object
dtypes: object(11)
memory usage: 126.0+ KB


In [43]:
bool_series = pd.isnull(top1000["Distributor"])
top1000[bool_series].head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Runtime,Genre,Rating,Distributor,Cast,Director
286,Get Out (2017),"$176,196,665",NaN,"$4,500,000","February 18, 2016\n (South Korea)",1 hr 44 min,Horror\n \n Mystery\n \n T...,R,NaN,Daniel Kaluuya\n \n Chris Washington...,Jordan Peele\n \n Director


In [44]:
top1000.loc[286, ["Distributor"]] = "Universal Pictures"

In [45]:
top1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1000 non-null   object
 1   Domestic_Gr  1000 non-null   object
 2   Dom_Open     969 non-null    object
 3   Budget       807 non-null    object
 4   Rel_Date     1000 non-null   object
 5   Runtime      1000 non-null   object
 6   Genre        1000 non-null   object
 7   Rating       1000 non-null   object
 8   Distributor  1000 non-null   object
 9   Cast         1000 non-null   object
 10  Director     1000 non-null   object
dtypes: object(11)
memory usage: 126.0+ KB


In [46]:
top1000.to_pickle("top1000.pkl")
top1000 = pd.read_pickle("top1000.pkl")